<a id="top"></a>
# Защита персональных данных клиентов
<h4 align="right">Спринт 9 | Когорта ДС13 | Артур Урусов</h4>

<blockquote>
<p>Вам нужно защитить данные клиентов страховой компании «Хоть потоп». Разработайте такой метод преобразования данных, чтобы по ним было сложно восстановить персональную информацию. Обоснуйте корректность его работы.
<p>
<p>Нужно защитить данные, чтобы при преобразовании качество моделей машинного обучения не ухудшилось. Подбирать наилучшую модель не требуется.
</blockquote>

## Содержание

0. **[Подготовка](#0)**

    0.1. [Импорт библиотек](#0-1)
    
    0.2. [Настройка окружения](#0-2)


1. **[Загрузка данных](#1)**


2. **[Умножение матриц](#2)**


3. **[Алгоритм преобразования](#3)**


4. **[Проверка алгоритма](#4)**

    4.1. [Функции алгоритма преобразования](#4-1)
    
    4.2. [Проверка результатов работы алгоритма](#4-2)
    

5. **[Результаты исследования](#5)**

    5.1. [Общие выводы](#5-1)
    
    5.2. [Чек-лист готовности проекта](#5-2)

<a id="0"></a>
## Этап 0. Подготовка

<a id="0-1"></a>
### Шаг 0.1 Импорт библиотек

In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression

[к началу шага](#0-1) | [к началу этапа](#0) | [к началу страницы](#top)

<a id="0-2"></a>
### Шаг 0.2 Настройка окружения

In [2]:
STATE=42

[к началу шага](#0-2) | [к началу этапа](#0) | [к началу страницы](#top)

<a id="1"></a>
## Этап 1. Загрузка данных

Загрузим данные и сохраним их в переменную. Затем получим основную информацию о датафрейме и выведем на экран первые несколько строк:

In [3]:
df = pd.read_csv('/datasets/insurance.csv')
df.info()
df.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5000 entries, 0 to 4999
Data columns (total 5 columns):
Пол                  5000 non-null int64
Возраст              5000 non-null float64
Зарплата             5000 non-null float64
Члены семьи          5000 non-null int64
Страховые выплаты    5000 non-null int64
dtypes: float64(2), int64(3)
memory usage: 195.4 KB


,Пол,Возраст,Зарплата,Члены семьи,Страховые выплаты
0,1,41.0,49600.0,1,0
1,0,46.0,38000.0,1,1
2,0,29.0,21000.0,0,0
3,0,21.0,41700.0,2,0
4,1,28.0,26100.0,0,0


Наш датафрейм состоит из 5000 строк и 5 столбцов с признаками:
- Признаки: `пол`, `возраст` и `зарплата` застрахованного, количество `членов его семьи`.
- Целевой признак: количество `страховых выплат` клиенту за последние 5 лет.

Данные не содержат пропущенных значений, а предобработка данных в целях текущего исследования не требуется, поэтому сразу перейдём к следующему этапу.

[к началу этапа](#1) | [к началу страницы](#top)

<a id="2"></a>
## Этап 2. Умножение матриц

Обозначения:

- $X$ — матрица признаков (нулевой столбец состоит из единиц)

- $y$ — вектор целевого признака

- $P$ — матрица, на которую умножаются признаки

- $w$ — вектор весов линейной регрессии (нулевой элемент равен сдвигу)

Основные формулы

Предсказания:

$
a = Xw
$

Задача обучения:

$
w = \arg\min_w MSE(Xw, y)
$

Формула обучения:

$
w = (X^T X)^{-1} X^T y
$

Нам необходимо доказать, что умножение матрицы признаков на квадратную обратимую матрицу не изменит качество предсказания целевого признака. Таким образом у нас получается два предсказания &mdash; для оригинальной матрицы, и для преобразованной:

$$
\left\{\begin{matrix}
\begin{align*}
& a = Xw = XEw = XPP^{-1}w \\
& {a}' = {X}'{w}' = XP{w}'
\end{align*}
\end{matrix}\right. \\
$$

Допустим, что предсказания равны, тогда:

$$
\begin{eqnarray}
a &=& {a}' \\
XPP^{-1}w &=& XP{w}',\ где\ X\ \neq \Theta, P \neq \Theta
\end{eqnarray}
$$

$X$ и $P$ не могут быть нулевыми матрицами. $X$ не является нулевой матрицей, так как первый столбец $X$ состоит из единиц. А $P$ не может быть нулевой матрицей, так как является обратимой матрицей. Таким образом, наше равенство сводится к:

$$
P^{-1}w = {w}'
$$

Распишем формулы обучения для обоих случаев:

$$
\left\{\begin{matrix}
\begin{align*}
& w = (X^{T}X)^{-1}X^{T}y \\ 
& {w}' = ((XP)^{T}XP)^{-1}(XP)^{T}y
\end{align*}
\end{matrix}\right. \\
$$

И подставим их в равенство:

$$
P^{-1}(X^{T}X)^{-1}X^{T}y = ((XP)^{T}XP)^{-1}(XP)^{T}y
$$

Воспользуемся свойствами $(AB)^{T} = B^{T}A^{T}$ и $(AB)^{-1} = B^{-1}A^{-1}$ (только для двух квадратных обратимых матриц):

$$
\begin{eqnarray}
P^{-1}(X^{T}X)^{-1}X^{T}y &=& (P^{T}X^{T}XP)^{-1}P^{T}X^{T}y \\
P^{-1}(X^{T}X)^{-1}X^{T}y &=& (X^{T}XP)^{-1}(P^{T})^{-1}P^{T}X^{T}y \\
P^{-1}(X^{T}X)^{-1}X^{T}y &=& P^{-1}(X^{T}X)^{-1}X^{T}y
\end{eqnarray}
$$

Приходим к тому, что левая и правая части равны, таким образом:

$$
\begin{eqnarray}
XPP^{-1}w &=& XP{w}' \\
a &=& {a}' \\
MSE\ (a,\ y) &=& MSE\ ({a}',\ y)
\end{eqnarray}
$$

Следовательно, предсказания модели на оригинальной и преобразованной выборках будут идентичными, а качество предсказания от преобразования не изменится.

[к началу этапа](#2) | [к началу страницы](#top)

<a id="3"></a>
## Этап 3. Алгоритм преобразования

Алгоритм шифрования достаточно простой и состоит из обычного линейного преобразования. Оригинальный массив данных с признаками $X$ размерности $(m\ x\ n)$ мы домножаем на случайную квадратную обратимую матрицу $P$ размерности $(n\ x\ n)$, которую подбираем с помощью генератора случайных чисел:

$$
{X}' = XP
$$

Как уже было доказано выше, такое преобразование признаков не влияет на качество предсказания целевого признака. Далее, как обычно, разделяем данные на обучающую и тестовую выборки и обучаем модель на тренировочной выборке. Задача обучения остаётся такой же, признаки изменены, но значения целевого признака остались неизменными:

$$
{w}' = \arg\min_{{w}'} MSE({X}'{w}', y)
$$

Формула обучения остаётся подобной, однако слегка усложняется дополнительной матрицей:

$$
{w}' = ((XP)^{T}XP)^{-1}(XP)^{T}y
$$

После этого получим предсказания:

$$
{a}' = {X}'{w}' = XP((XP)^{T}XP)^{-1}(XP)^{T}y
$$

И оценим качество предсказаний с помощью функции MSE:
$$
MSE\ ({a}',\ y)
$$

Аналогичные манипуляции проделаем и с тестовой выборкой зашифрованных данных. Такие же операции произведем и для обычных, незашифрованных данных, после чего сравним результаты для зашифрованных и незашифрованных данных. С учётом доказательства, полученного выше, результаты для зашифрованной и незашифрованной выборок должны быть идентичными.

Кроме того, мы можем восстановить зашифрованные данные с помощью умножения преобразованной матрицы на матрицу $P^{-1}$, обратную $P$:

$$
{X}'P^{-1} = XPP^{-1} = XE = X
$$


Таким образом, на следующем шаге нам следует:
1. Создать случайную матрицу $P$ размерности $(n\ x\ n)$ для матрицы с данными $X$ размерности $(m\ x\ n)$.

    
2. Сразу проверяем полученную матрицу на обратимость.

    2.1. Если матрица обратима, идём дальше.
    
    2.2. Если матрица необратима, генерируем новые матрицы, до тех пор пока не получим обратимую матрицу или пока не достигнем лимита итераций.
    
    
3. Сохранаяем пару матриц $P,\ P^{-1}$ как ключи для шифровки/ дешифровки.

    
4. Преобразовываем данные с помощью умножения матрицы $X$ на матрицу $P$.

    
5. Проверяем способность восстановить данные с помощью умножения на матрицу $P^{-1}$.

    
6. Сравниваем метрики для линейной регрессии на оригинальных и преобразованных данных.

    
7. Делаем вывод.

[к началу этапа](#3) | [к началу страницы](#top)

<a id="4"></a>
## Этап 4. Проверка алгоритма

<a id="4-1"></a>
### Шаг 4.1 Функции алгоритма преобразования

Напишем несколько функций:
- `matrix_passkey` для генерирования случайных обратимых матриц и обратных к ним матриц;
- `matrix_encoder` для шифрования исходных данных;
- `matrix_decoder` для восстановления исходных данных;
- `decryption_accuracy` для оценки качества восстановления первоначальных данных.

In [4]:
def matrix_passkey(shape, max_iterations=10):
    rng = np.random.default_rng()
    for _ in range(max_iterations):
        P = rng.random(shape)
        try:
            P_inverted = np.linalg.inv(P)
            return P, P_inverted
        except LinAlgError:
            continue
    raise LinAlgError('За указанное количество итераций не удалось найти обратимую матрицу указанной размерности')
    
def matrix_encoder(data, max_iterations=10):
    encryption_key, decryption_key = matrix_passkey((data.shape[1], data.shape[1]), max_iterations)
    return data.dot(encryption_key), decryption_key

def matrix_decoder(data, decryption_key):
    return data.dot(decryption_key)

def decryption_accuracy(original_data, decrypted_data):
    n = 0
    while (np.array(original_data.round(n)) == np.array(decrypted_data.round(n))).all():
        n += 1
    else:
        print(f'Значения совпадают с точностью до {n - 1} знаков после запятой.')

[к началу шага](#4-1) | [к началу этапа](#4) | [к началу страницы](#top)

<a id="4-2"></a>
### Шаг 4.2 Проверка результатов работы алгоритма

Теперь разделим данные на `features` и `target`:

In [5]:
features = ['Пол', 'Возраст', 'Зарплата', 'Члены семьи']
target = 'Страховые выплаты'
X = df[features]
y = df[target]

Также зашифруем признаки, а заодно и расшифруем обратно:

In [6]:
X_encrypted, decrypter = matrix_encoder(X)
X_decrypted = matrix_decoder(X_encrypted, decrypter)

Сразу проверим качество дешифровки:

In [7]:
decryption_accuracy(X, X_decrypted)

Значения совпадают с точностью до 9 знаков после запятой.


Достаточно точное восстановление данных для наших потребностей.

Теперь разобьём данные на тренировочную и тестовую выборки:

In [8]:
X_train, X_test, y_train, y_test = train_test_split(
    X, y, random_state=STATE)
X_train_encrypted, X_test_encrypted, y_train_encrypted, y_test_encrypted = train_test_split(
    X_encrypted, y, random_state=STATE)

Обучим модель на оригинальных данных и проверим результаты на тренировочной и тестовой выборках как оригинальных данных, так и зашифрованных, а потом обучим модель только на зашифрованных данных и проверим результаты:

In [9]:
lrm = LinearRegression()
lrm.fit(X_train, y_train)
print('Модель, обученная на оригинальной выборке.')
print(f'Результаты на оригинальной тренировочной выборке: {lrm.score(X_train, y_train):,.4f}')
print(f'Результаты на оригинальной тестовой выборке: {lrm.score(X_test, y_test):,.4f}')
print(f'Результаты на преобразованной тренировочной выборке: {lrm.score(X_train_encrypted, y_train_encrypted):,.4f}')
print(f'Результаты на преобразованной тестовой выборке: {lrm.score(X_test_encrypted, y_test_encrypted):,.4f}')

lrm_encrypted = LinearRegression()
lrm_encrypted.fit(X_train_encrypted, y_train_encrypted)
print('\nМодель, обученная на преобразованной выборке.')
print(f'Результаты на преобразованной тренировочной выборке: {lrm_encrypted.score(X_train_encrypted, y_train_encrypted):,.4f}')
print(f'Результаты на преобразованной тестовой выборке: {lrm_encrypted.score(X_test_encrypted, y_test_encrypted):,.4f}')

Модель, обученная на оригинальной выборке.
Результаты на оригинальной тренировочной выборке: 0.4244
Результаты на оригинальной тестовой выборке: 0.4255
Результаты на преобразованной тренировочной выборке: -578,635.7400
Результаты на преобразованной тестовой выборке: -489,682.1892

Модель, обученная на преобразованной выборке.
Результаты на преобразованной тренировочной выборке: 0.4244
Результаты на преобразованной тестовой выборке: 0.4255


Как видим, результаты моделей обученных на оригинальной и преобразованной выборках при предсказаниях на соответствующих выборках практически не отличаются. То есть, использование обратимой матрицы для преобразования данных действительно не влияет на точность предсказания модели. Однако, под преобразованную выборку модель нужно обучать отдельно, иначе результаты будут очень плохие (как видно на нашей модели).

[к началу шага](#4-2) | [к началу этапа](#4) | [к началу страницы](#top)

<a id="5"></a>
## Этап 5. Проверка алгоритма

<a id="5-1"></a>
### Шаг 5.1 Общие выводы

В рамках проведенного исследования мы постарались найти способ защиты персональных данных клиентов, который не влияет на качество предсказания целевого признака моделями машинного обучения.

Для работы нам предоставили датасет из 5000 строк и 5 столбцов с признаками клиентов.

Сначала мы изучили датасет, не нашли пропусков и других проблем и сразу перешли к следующему шагу.

Первоначальной задачей стал поиск ответа на вопрос «Признаки умножают на обратимую матрицу. Изменится ли качество линейной регрессии?». Мы предположили, что качество не изменится, и, приравняв предсказания двух выборок, доказали, что качество предсказания не будет зависеть от умножения на обратимую матрицу.

Далее мы предложили алгоритм модели, которая преобразует матрицу и предсказывает на основе преобразованных данных целевой признак.

На заключительном этапе мы проверили свои предположения на практике, оценили подобие качества предсказания на оригинальной и преобразованной выборках, а также проверили качество восстановления данных с помощью обратной матрицы.

В заключение можно сказать, что с точки зрения предсказательных возможностей моделей машинного обучения, способ защиты данных с помощью умножения на матрицу является допустимым. К преимуществам данного подхода можно отнести, в первую очередь, ощутимую экономию ресурсов по сравнению с некоторыми известными алгоритмами шифрования.

Однако, с увеличением количества признаков, проверка на вырожденность и другие расчёты могут стать ресурсозатратными. Кроме того, необходимо проверить и кроиптографическую устойчивость такого метода защиты данных, перед тем как принимать решение о практическом применении.

<a id="5-2"></a>
### Шаг 5.2 Чек-лист готовности проекта

- [x]  Jupyter Notebook открыт
- [x]  Весь код выполняется без ошибок
- [x]  Ячейки с кодом расположены в порядке исполнения
- [x]  Выполнен шаг 1: данные загружены
- [x]  Выполнен шаг 2: получен ответ на вопрос об умножении матриц
    - [x]  Указан правильный вариант ответа
    - [x]  Вариант обоснован
- [x]  Выполнен шаг 3: предложен алгоритм преобразования
    - [x]  Алгоритм описан
    - [x]  Алгоритм обоснован
- [x]  Выполнен шаг 4: алгоритм проверен
    - [x]  Алгоритм реализован
    - [x]  Проведено сравнение качества моделей до и после преобразования

[к началу шага](#5-2) | [к началу этапа](#5) | [к началу страницы](#top)